<a href="https://colab.research.google.com/github/afshalliaquat/Sentimental_Analysis_NaiveBayes.ipynb/blob/main/Sentimental_Analysis_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.txt',sep = ';',header=None,names= ['text','emotions'])

In [3]:
unique_emotions = df['emotions'].unique()
emotion_no = {}
i=0

for emo in unique_emotions:
  emotion_no[emo]=i
  i+=1

df['emotions'] = df['emotions'].map(emotion_no)

In [4]:
df['text'] = df['text'].apply(lambda x:x.lower())

In [5]:
import string

In [6]:
def remove_punc(text):
  return text.translate(str.maketrans('','',string.punctuation))

In [7]:
df['text'] = df['text'].apply(remove_punc)

In [8]:
def remove_no(text):
  string = ''
  for i in text:
    if not i.isdigit():
      string += i
  return string


In [9]:
df['text'] = df['text'].apply(remove_no)

In [10]:
def remove_emoji(text):
  string = ''
  for i in text:
    if i.isascii():
      string+=i
  return string

In [11]:
df['text'] = df['text'].apply(remove_emoji)

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [13]:
nltk.download('stopwords')
nltk.download('punkt')       # for word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove(txt):
  words = txt.split()
  cleaned = []
  for i in words:
    if not i in stop_words:
      cleaned.append(i)
  return ' '.join(cleaned)

In [16]:
df['text'] = df['text'].apply(remove)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [18]:
X = df['text']
y= df['emotions']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer  #Bag of words (only 0 and 1s)

In [21]:
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [23]:
from sklearn.svm import SVC

model_svm_tfid = SVC()
model_svm_tfid.fit(X_train_vector, y_train)
y_pred_tfid = model_svm_tfid.predict(X_test_vector)
accuracy_score(y_test,y_pred_tfid)

0.85125

In [24]:
Parameter_SVM = {
    'C': [0.1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
}

In [25]:
# SVM_para_tfid = RandomizedSearchCV(estimator=model_svm_tfid,n_iter=5,cv=5,param_distributions=Parameter_SVM)
# SVM_para_tfid.fit(X_train_vector,y_train)
# new_model_tfid =SVM_para_tfid.best_estimator_
# y_pre_tfid = new_model_tfid.predict(X_test_vector)
# accuracy_score(y_test,y_pre_tfid)

In [26]:
from sklearn.svm import SVC

model_svm_bow = SVC()
model_svm_bow.fit(X_train_bow, y_train)
y_pred_bow = model_svm_bow.predict(X_test_bow)
accuracy_score(y_test,y_pred_bow)

0.8225

In [27]:
# SVM_para_bow = RandomizedSearchCV(estimator=model_svm_bow,n_iter=5,cv=5,param_distributions=Parameter_SVM)
# SVM_para_bow.fit(X_train_vector,y_train)
# new_model_svm_bow =SVM_para_bow.best_estimator_
# y_pre_bow = new_model_svm_bow.predict(X_test_vector)
# accuracy_score(y_test,y_pre_bow)

In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_tfid))
print(classification_report(y_test, y_pred_bow))


              precision    recall  f1-score   support

           0       0.90      0.93      0.91       946
           1       0.91      0.81      0.85       427
           2       0.89      0.55      0.68       296
           3       0.84      0.51      0.64       113
           4       0.85      0.75      0.80       397
           5       0.79      0.96      0.87      1021

    accuracy                           0.85      3200
   macro avg       0.86      0.75      0.79      3200
weighted avg       0.86      0.85      0.85      3200

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       946
           1       0.90      0.72      0.80       427
           2       0.88      0.51      0.65       296
           3       0.84      0.45      0.59       113
           4       0.84      0.70      0.77       397
           5       0.74      0.96      0.83      1021

    accuracy                           0.82      3200
   macro avg       0.85